In [2]:
#import libraries
import os
import json

from zipfile import ZipFile
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
#kaggle API
kaggle_dictionary = json.load(open("kaggle.json"))

In [4]:
# setup kaggle 
os.environ["KAGGLE_USERNAME"] = kaggle_dictionary["username"]
os.environ["KAGGLE_KEY"] = kaggle_dictionary["key"]

In [5]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other




  0%|          | 0.00/25.7M [00:00<?, ?B/s]
  4%|3         | 1.00M/25.7M [00:00<00:12, 2.03MB/s]
  8%|7         | 2.00M/25.7M [00:00<00:07, 3.27MB/s]
 12%|#1        | 3.00M/25.7M [00:00<00:05, 4.65MB/s]
 19%|#9        | 5.00M/25.7M [00:01<00:03, 6.87MB/s]
 27%|##7       | 7.00M/25.7M [00:01<00:02, 8.16MB/s]
 35%|###5      | 9.00M/25.7M [00:01<00:01, 8.97MB/s]
 43%|####2     | 11.0M/25.7M [00:01<00:01, 9.52MB/s]
 51%|#####     | 13.0M/25.7M [00:01<00:01, 9.87MB/s]
 54%|#####4    | 14.0M/25.7M [00:01<00:01, 9.95MB/s]
 58%|#####8    | 15.0M/25.7M [00:02<00:01, 9.85MB/s]
 66%|######6   | 17.0M/25.7M [00:02<00:00, 10.4MB/s]
 74%|#######3  | 19.0M/25.7M [00:02<00:00, 10.5MB/s]
 82%|########1 | 21.0M/25.7M [00:02<00:00, 10.6MB/s]
 89%|########9 | 23.0M/25.7M [00:02<00:00, 10.6MB/s]
 97%|#########7| 25.0M/25.7M [00:02<00:00, 10.6MB/s]
100%|##########| 25.7M/25.7M [00:03<00:00, 8.86MB/s]


In [12]:
# unzip the dataset file
with ZipFile("imdb-dataset-of-50k-movie-reviews.zip", "r") as zip_ref:
  zip_ref.extractall()

In [13]:
data = pd.read_csv("IMDB Dataset.csv")

In [14]:
data.shape

(50000, 2)

In [15]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [16]:
data["sentiment"].value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

In [17]:
data.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace=True)
data.head()

C:\Users\ARUL KUMARAN\AppData\Local\Temp\ipykernel_15592\1714321766.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace=True)


,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [18]:
data["sentiment"].value_counts()

sentiment
1    25000
0    25000
Name: count, dtype: int64

In [19]:
# split data into training data and test data
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

In [20]:
print(train_data.shape)
print(test_data.shape)

(40000, 2)
(10000, 2)


In [21]:
# Tokenize text data
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_data["review"])
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data["review"]), maxlen=200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data["review"]), maxlen=200)

In [22]:
print(X_train)

[[1935    1 1200 ...  205  351 3856]
 [   3 1651  595 ...   89  103    9]
 [   0    0    0 ...    2  710   62]
 ...
 [   0    0    0 ... 1641    2  603]
 [   0    0    0 ...  245  103  125]
 [   0    0    0 ...   70   73 2062]]


In [23]:
print(X_test)

[[   0    0    0 ...  995  719  155]
 [  12  162   59 ...  380    7    7]
 [   0    0    0 ...   50 1088   96]
 ...
 [   0    0    0 ...  125  200 3241]
 [   0    0    0 ... 1066    1 2305]
 [   0    0    0 ...    1  332   27]]


In [24]:
Y_train = train_data["sentiment"]
Y_test = test_data["sentiment"]

In [25]:
print(Y_train)

39087    0
30893    0
45278    1
16398    0
13653    0
        ..
11284    1
44732    1
38158    0
860      1
15795    1
Name: sentiment, Length: 40000, dtype: int64


In [26]:
# build the model

model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=200))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation="sigmoid"))

C:\Users\ARUL KUMARAN\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [27]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [28]:
# compile the model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [29]:
# evaluate

model.fit(X_train, Y_train, epochs=5, batch_size=64, validation_split=0.2)

Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 177s 348ms/step - accuracy: 0.7876 - loss: 0.4595 - val_accuracy: 0.8384 - val_loss: 0.3742
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 170s 339ms/step - accuracy: 0.8487 - loss: 0.3625 - val_accuracy: 0.8363 - val_loss: 0.3710
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 177s 353ms/step - accuracy: 0.8739 - loss: 0.3081 - val_accuracy: 0.8646 - val_loss: 0.3450
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 180s 359ms/step - accuracy: 0.8995 - loss: 0.2536 - val_accuracy: 0.8735 - val_loss: 0.3064
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 176s 352ms/step - accuracy: 0.9097 - loss: 0.2332 - val_accuracy: 0.8792 - val_loss: 0.3290


In [30]:
#evaluate
loss, accuracy = model.evaluate(X_test, Y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 9s 27ms/step - accuracy: 0.8795 - loss: 0.3193
Test Loss: 0.3192632794380188
Test Accuracy: 0.8794999718666077


In [31]:
#predictive system
def predict_sentiment(review):
  # tokenize and pad the review
  sequence = tokenizer.texts_to_sequences([review])
  padded_sequence = pad_sequences(sequence, maxlen=200)
  prediction = model.predict(padded_sequence)
  sentiment = "positive" if prediction[0][0] > 0.5 else "negative"
  return sentiment

In [32]:
# example usage
new_review = "This movie was fantastic. I loved it."
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 481ms/step
The sentiment of the review is: positive
